# 5. Searching KItems with the SDK

In this tutorial we see how to search existing Kitems

### 5.1. Setting up
Before you run this tutorial: make sure to have access to a DSMS-instance of your interest, alongwith with installation of this package and have establised access to the DSMS through DSMS-SDK (refer to [Connecting to DSMS](../dsms_sdk.md#connecting-to-dsms))


Now let us import the needed classes and functions for this tutorial.

In [1]:
from dsms import DSMS, KItem

Now source the environmental variables from an `.env` file and start the DSMS-session.

In [2]:
dsms = DSMS(env=".env")

### 5.2. Searching for KItems

In this section, we would like to search for specfic KItems we created in the DSMS.

For this purpose, we will firstly create some KItems and apply the `search`-method on the `DSMS`-object later on in order to find them again in the DSMS.

We also want to demonstrate here, that we can link KItems to each other in order to find e.g. a related item of type `DatasetCatalog`. For this strategy, we are using the `linked_kitems`- attribute and the `id` of the item which we would like to link.

The procedure looks like this:

In [3]:
item1 = KItem(
    name="Machine-1",
    ktype_id=dsms.ktypes.TestingMachine,
    custom_properties={"Producer": "TestingLab GmBH",
                       "Room Number": "A404",
                       "Description": "Bending Test Machine"
                       }
)

item2 = KItem(
    name="Machine-2",
    ktype_id=dsms.ktypes.TestingMachine,
    custom_properties={"Producer": "StressStrain GmBH",
                       "Room Number": "B500",
                       "Description": "Compression Test Machine"
                       }
)

item3 = KItem(
    name="Specimen-1", 
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item1],
    custom_properties={"Geometry": "Cylindrical 150mm x 20mm x 40mm",
                       "Material": "Concrete",
                       "Project ID": "ConstructionProject2024"
    }

)
item4 = KItem(
    name="Specimen-2",
    ktype_id=dsms.ktypes.Specimen,
    linked_kitems=[item2],
    custom_properties={"Geometry": "Rectangular 200mm x 30mm x 20mm",
                       "Material": "Metal",
                       "Project ID": "MetalBlenders2024"
                       }
)

item5 = KItem(
    name="Research Institute ABC",
    ktype_id=dsms.ktypes.Organization,
    linked_kitems=[item1],
    annotations=[
        {
            "iri": "www.researchBACiri.org/foo",
            "name": "research ABC Institute",
            "namespace": "research",
        }
    ],
)

dsms.commit()


#### <p style="color:red;">Note : Here in this tutorial, we use dsms.search with `limit=1` to maintain readability but the user can adjust the variable `limit` as per requirement. </p>


Now, we are apply to search for e.g. kitems of type `TestingMachine`:

In [4]:
for result in dsms.search(ktypes=[dsms.ktypes.TestingMachine], limit=1):
    print(result.hit)
    print("\n")

KItem(

	name = Machine-1, 

	id = 0f49b0ad-2a98-4c10-8440-fd3c37363646, 

	ktype_id = testing-machine, 

	in_backend = True, 

	slug = machine-1-0f49b0ad, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [
		
			id: 075709da-5481-4cb4-a8ee-c18db34ce9d3
			name: Specimen-1
			slug: specimen-1-075709da
			ktype_id: specimen
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: 0f49b0ad-2a98-4c10-8440-fd3c37363646
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'sampletype': None, 'sampleinformation': None, 'sampleproductionprocess': None, 'Geometry': 'Cylindrical 150mm x 20mm x 40mm', 'Material': 'Concrete', 'Project ID': 'ConstructionProject2024'}
			created_at: 2024-08-19T18:26:00.499708
			updated_at: 2024-08-19T18:26:00.499708
		
	], 

	affiliations = [], 

	authors = [
		{
			user_id: 7f

... and for all of type `Organization` and `DatasetCatalog`:

In [5]:
for result in dsms.search(ktypes=[dsms.ktypes.Organization, dsms.ktypes.DatasetCatalog], limit=1):
    print(result.hit)
    print("fuzziness: ", result.fuzzy)
    print("\n")
    

KItem(

	name = Research Institute ABC, 

	id = 21aa50c3-5ec2-4ac3-aba8-69071a4287e2, 

	ktype_id = organization, 

	in_backend = True, 

	slug = researchinstituteabc-21aa50c3, 

	annotations = [], 

	attachments = [], 

	linked_kitems = [], 

	affiliations = [], 

	authors = [
		{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}
	], 

	avatar_exists = False, 

	contacts = [], 

	created_at = 2024-08-19 18:26:00.740761, 

	updated_at = 2024-08-19 18:26:00.740761, 

	external_links = [], 

	kitem_apps = [], 

	summary = None, 

	user_groups = [], 

	custom_properties = None, 

	dataframe = None, 

	rdf_exists = False
)
fuzziness:  False




... or for all of type `Dataset` with `Specimen-1` in the name:

In [6]:
for result in dsms.search(query="Specimen-1", ktypes=[dsms.ktypes.Dataset], limit=1):
    print(result.hit)
    print("\n")

... and for all of type `Organization` with the annotation `www.researchBACiri.org/foo`:

In [7]:
for result in dsms.search(
          ktypes=[dsms.ktypes.Organization], annotations=["www.researchBACiri.org/foo"], limit=1
    ):
    print(result.hit)
    print("\n")

KItem(

	name = Research Institute ABC, 

	id = 84c089e6-feab-4ba6-a934-527594e504eb, 

	ktype_id = organization, 

	in_backend = True, 

	slug = researchinstituteabc-84c089e6, 

	annotations = [
		{
			iri: www.researchBACiri.org/foo,
			name: research ABC Institute,
			namespace: research,
			description: None
		}
	], 

	attachments = [], 

	linked_kitems = [
		
			id: 694561e9-df15-4c71-ae71-abd91df3ec23
			name: Machine-1
			slug: machine-1-694561e9
			ktype_id: testing-machine
			summary: None
			avatar_exists: False
			annotations: []
			linked_kitems: [{
			id: 83672cdb-7584-4dd7-9b6e-d10574f1adf7
		}, {
			id: 84c089e6-feab-4ba6-a934-527594e504eb
		}]
			external_links: []
			contacts: []
			authors: [{
			user_id: 7f0e5a37-353b-4bbc-b1f1-b6ad575f562d
		}]
			linked_affiliations: []
			attachments: []
			user_groups: []
			custom_properties: {'Producer': 'TestingLab GmBH', 'Room Number': 'A404', 'Description': 'Bending Test Machine'}
			created_at: 2024-08-19T18:26:08.898435
		

Clean up the DSMS from the tutortial:

In [8]:
del dsms[item1]
del dsms[item2]
del dsms[item3]
del dsms[item4]
del dsms[item5]

dsms.commit()